In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from collections import defaultdict
from joblib import Parallel, delayed

import sqlite3
import sys
import time
import math
#import tqdm
from tqdm.auto import tqdm
import datetime
import os
import pickle
from pathlib import Path

from glicko2 import Player
import multiprocessing

import itertools

tqdm.pandas()

if os.path.exists('/workspace/data'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data/'
else:
    data_path = '../data/'

## Loading SQLite Database into Pandas DataFrames

The following code connects to an SQLite database (`melee_player_database.db`) and converts each table within the database into a pandas DataFrame. The DataFrames will be stored in a dictionary, where each key corresponds to the table name with `_df` appended, and the values are the respective DataFrames.

### Steps:

1. **Database Connection**: We use the `sqlite3` library to connect to the SQLite database file.
2. **Retrieve Table Names**: A query retrieves all the table names in the database.
3. **Convert Tables to DataFrames**: For each table:
   - The table is loaded into a pandas DataFrame using `pd.read_sql()`.
   - We check each column to see if any data is JSON-formatted (lists or dictionaries). If so, we convert these columns from strings into their corresponding Python objects using `json.loads()`.
4. **Store DataFrames**: The DataFrames are stored in a dictionary, where the key is the table name with a `_df` suffix, and the value is the DataFrame.
5. **Database Connection Closed**: Once all tables are loaded into DataFrames, the database connection is closed.

### Example:
If the database contains a table named `players`, the corresponding DataFrame will be stored in the dictionary with the key `players_df`, and can be accessed as:

```python
players_df = dfs['players_df']


In [2]:
# Function to get the table names
def get_table_names(conn):
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    return pd.read_sql(query, conn)['name'].tolist()

# Function to load tables into DataFrames
def load_tables_to_dfs(conn):
    table_names = get_table_names(conn)
    dataframes = {}
    
    for table in table_names:
        # Load table into a DataFrame
        df = pd.read_sql(f"SELECT * FROM {table}", conn)
        
        # Detect and convert JSON formatted columns (if any)
        for col in df.columns:
            # Check if any entry in the column is a valid JSON (list or dictionary)
            if df[col].apply(lambda x: isinstance(x, str)).all():
                try:
                    # Try parsing the column as JSON
                    df[col] = df[col].apply(lambda x: json.loads(x) if pd.notnull(x) else x)
                except (json.JSONDecodeError, TypeError):
                    # If it fails, skip the column
                    pass
        
        # Store the DataFrame with table name + '_df'
        dataframes[f"{table}_df"] = df
        
    return dataframes

if os.path.exists(data_path + 'dfs_dict.pkl'):
    cell_has_run = True
    # Load the dictionary of DataFrames from the pickle
    with open(data_path + 'dfs_dict.pkl', 'rb') as f:
        dfs = pickle.load(f)
# Check if the flag variable exists in the global scope so that this code does not run twice
if 'cell_has_run' not in globals():
    path = data_path + "melee_player_database.db"
    
    # Connect to the database
    conn = sqlite3.connect(path)

    # Convert each table into a DataFrame
    dfs = load_tables_to_dfs(conn)

    # Close the connection
    conn.close()

    # Now, you have a dictionary 'dfs' where each key is the table name with '_df' suffix and value is the corresponding DataFrame.
    # For example, to access the DataFrame for a table called 'players':
    # players_df = dfs['players_df']

    dfs['tournament_info_df']['start'] = pd.to_datetime(dfs['tournament_info_df']['start'], unit='s')
    dfs['tournament_info_df']['end'] = pd.to_datetime(dfs['tournament_info_df']['end'], unit='s')

    
    # Set the flag to indicate that the cell has been run
    cell_has_run = True

### Here we adjust the data types of the dataframes so that they are the correct type. (This will be updated as needed.)

In [3]:
dfs['sets_df']['best_of'] = dfs['sets_df']['best_of'].fillna(0).astype(int) 

### Here we make dataframes that we will use and print the head.

The integers in 'characters' count the number of games the player has played that character. (We verify this for Zain below.)

In [4]:
players_df = dfs['players_df']
players_df.head()

,game,player_id,tag,all_tags,prefixes,social,country,state,region,c_country,c_state,c_region,placings,characters,alias
0,melee,Rishi,Rishi,[Rishi],[],{'twitter': []},,,,None,None,None,[{'key': 'mdva-invitational-2017-(challonge-mi...,,None
1,melee,15634,lloD,"[lloD, VGz | lloD, Llod]",[],{'twitter': ['lloD74']},United States,VA,,US,CA,Laurel,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/peach': 1089, 'melee/falco': 1, 'melee...",None
2,melee,6126,Zain,"[Zain, DontTestMe]",[PG],{'twitter': ['PG_Zain']},United States,VA,,US,CA,Los Angeles,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/marth': 1065, 'melee/pichu': 1, 'melee...",DontTestMe
3,melee,Chu,Chu,[Chu],[],{'twitter': []},,,,None,None,None,[{'key': 'mdva-invitational-2017-(challonge-mi...,,None
4,melee,5620,Junebug,"[Junebug, LS | VGz Junebug]",[],{'twitter': ['arJunebug']},United States,VA,,US,VA,Richmond,[{'key': 'mdva-invitational-2017-(challonge-mi...,"{'melee/sheik': 46, 'melee/falco': 4, 'melee/g...",None


In [5]:
ranking_df = dfs['ranking_df']
ranking_df.head()

,game,ranking_name,priority,region,seasons,tournaments,icon
0,melee,SSBMRank,0,world,"[2015, 2016, 2017, 2018, 2019]",[],miom


In [6]:
ranking_seasons_df = dfs['ranking_seasons_df']
ranking_seasons_df.head()

,game,ranking_name,season,start,end,total,by_id,by_placing,final,name
0,melee,SSBMRank,2015,1420070400,1451606399,100,"{'6189': 1, '1004': 2, '4465': 3, '1000': 4, '...","{'1': '6189', '2': '1004', '3': '4465', '4': '...",0,None
1,melee,SSBMRank,2016,1451606400,1483228799,100,"{'6189': 1, '1004': 2, '1000': 3, '1003': 4, '...","{'1': '6189', '2': '1004', '3': '1000', '4': '...",0,None
2,melee,SSBMRank,2017,1483228800,1514764799,100,"{'1004': 1, '6189': 2, '1000': 3, '1003': 4, '...","{'1': '1004', '2': '6189', '3': '1000', '4': '...",0,None
3,melee,SSBMRank,2018,1514793600,1546329600,100,"{'1004': 1, '6189': 2, '4465': 3, '15990': 4, ...","{'1': '1004', '2': '6189', '3': '4465', '4': '...",0,None
4,melee,SSBMRank,2019,1546329600,1577836800,100,"{'1004': 1, '4465': 2, '1000': 3, '16342': 4, ...","{'1': '1004', '2': '4465', '3': '1000', '4': '...",0,None


In [7]:
sets_df = dfs['sets_df']
print(f"{sets_df[sets_df['game_data'].apply(lambda x: len(x) > 0)].shape[0] / sets_df.shape[0]:0.01%} percent of sets have some game data)")

sets_df.head()

32.9% percent of sets have some game data)


,key,game,tournament_key,winner_id,p1_id,p2_id,p1_score,p2_score,location_names,bracket_name,bracket_order,set_order,best_of,game_data
0,104675843,melee,mdva-invitational-2017-(challonge-mirror),5620,5620,Chillin,3,1,"[R1, Round 1, Round 1]",,1,A,5,[]
1,104675844,melee,mdva-invitational-2017-(challonge-mirror),Aglet,15634,Aglet,2,3,"[R1, Round 1, Round 1]",,1,B,5,[]
2,104675845,melee,mdva-invitational-2017-(challonge-mirror),6126,6126,1097,3,0,"[R1, Round 1, Round 1]",,1,C,5,[]
3,104675846,melee,mdva-invitational-2017-(challonge-mirror),1069,Chu,1069,0,3,"[R1, Round 1, Round 1]",,1,D,5,[]
4,104675847,melee,mdva-invitational-2017-(challonge-mirror),Rishi,Jerry,Rishi,1,3,"[R1, Round 1, Round 1]",,1,E,5,[]


In [8]:
tournament_info_df = dfs['tournament_info_df']
tournament_info_df.head()

,game,key,cleaned_name,source,tournament_name,tournament_event,season,rank,start,end,country,state,city,entrants,placings,losses,bracket_types,online,lat,lng
0,melee,mdva-invitational-2017-(challonge-mirror),MDVA Invitational 2017 (Challonge Mirror),challonge,https://challonge.com/mdva_invitational_2017,,17,,2017-11-26 08:05:11,2017-11-26 08:48:09,US,VA,Fall's Church,10,"[[Rishi, 1], [15634, 3], [6126, 4], [Chu, 8], ...",{},b'{}',0,NaN,NaN
1,melee,s@sh7,S@SH7,challonge,https://challonge.com/sash7,,17,,2017-06-13 10:27:01,2017-06-13 10:27:01,US,MI,Ann Arbor,92,[],{},b'{}',0,NaN,NaN
2,melee,slippi-champions-league-week-1__melee-singles,Slippi Champions League Week 1,pgstats,slippi-champions-league-week-1,melee-singles,20,,2020-10-11 14:00:00,2020-10-11 14:00:00,,,,20,"[[1000, 1], [6126, 2], [4107, 3], [19554, 3], ...",{},b'{}',1,0.0,0.0
3,melee,slippi-champions-league-week-2__melee-singles,Slippi Champions League Week 2,pgstats,slippi-champions-league-week-2,melee-singles,20,,2020-10-18 14:00:00,2020-10-18 14:00:00,,,,20,"[[6126, 1], [4107, 2], [1000, 3], [19554, 3], ...",{},b'{}',1,0.0,0.0
4,melee,slippi-champions-league-week-3__melee-singles,Slippi Champions League Week 3,pgstats,slippi-champions-league-week-3,melee-singles,20,,2020-10-25 14:00:00,2020-10-25 14:00:00,,,,20,"[[6126, 1], [3359, 2], [19554, 3], [4107, 3], ...",{},b'{}',1,0.0,0.0


In [22]:
#sets_df.iloc[0]['p1_id']
#sets_df[(sets_df['p1_id']=='5620')]
print(len(sets_df[(sets_df['p1_id']=='5620') & (sets_df['p2_id']=='Chillin')]))
len(sets_df[(sets_df['p1_id']=='568920')])

1


0

In [9]:
# Code optimization by Dan
# Basically we want to replace this line in process_tournament with something more efficient:
#
#      tournament_sets_df = sets_df[sets_df['tournament_key'] == tournament_key]
#
# Instead, we can
# - Merge the tournament date info into ``sets_df``
# - Sort by date
# - Store the start/end positions of each tournament in a separate dictionary
# - Use tournament_sets_df = sets_df.iloc[start:end+1] instead.

sets_df = sets_df.merge(tournament_info_df[['key', 'start', 'end']], left_on='tournament_key', right_on='key', how='left')
sets_df = sets_df.drop(labels=['key_y'], axis='columns')
sets_df = sets_df.rename(columns={"key_x": "key"})
sets_df = sets_df.sort_values(by=['end', 'tournament_key']) # Just in case there are tournaments with the exact same end date

In [10]:
# A bit of data cleanup
# TODO: Rerun!
min_date = datetime.datetime(2015, 1, 1)
max_date = datetime.datetime(2024, 12, 31)

sets_df = sets_df[(sets_df['start'] >= min_date) & (sets_df['end'] >= min_date) & (sets_df['start'] <= max_date) & (sets_df['end'] <= max_date)]

In [30]:
sets_df.head()

,key,game,tournament_key,winner_id,p1_id,p2_id,p1_score,p2_score,location_names,bracket_name,bracket_order,set_order,best_of,game_data
0,104675843,melee,mdva-invitational-2017-(challonge-mirror),5620,5620,Chillin,3,1,"[R1, Round 1, Round 1]",,1,A,5,[]
1,104675844,melee,mdva-invitational-2017-(challonge-mirror),Aglet,15634,Aglet,2,3,"[R1, Round 1, Round 1]",,1,B,5,[]
2,104675845,melee,mdva-invitational-2017-(challonge-mirror),6126,6126,1097,3,0,"[R1, Round 1, Round 1]",,1,C,5,[]
3,104675846,melee,mdva-invitational-2017-(challonge-mirror),1069,Chu,1069,0,3,"[R1, Round 1, Round 1]",,1,D,5,[]
4,104675847,melee,mdva-invitational-2017-(challonge-mirror),Rishi,Jerry,Rishi,1,3,"[R1, Round 1, Round 1]",,1,E,5,[]


In [40]:
sets_df.iloc[0]
sets_df.columns

Index(['key', 'game', 'tournament_key', 'winner_id', 'p1_id', 'p2_id',
       'p1_score', 'p2_score', 'location_names', 'bracket_name',
       'bracket_order', 'set_order', 'best_of', 'game_data'],
      dtype='object')

In [36]:
# Example of game data. List of dictionaries.
sets_df[sets_df['game_data'].apply(lambda x: x != [])].iloc[0]['game_data']

[{'loser_char': 'melee/peach',
  'winner_score': 1,
  'winner_id': 1004,
  'loser_id': 1032,
  'winner_char': 'melee/jigglypuff',
  'loser_score': 0,
  'stage': 'Battlefield'},
 {'loser_char': 'melee/peach',
  'winner_score': 2,
  'winner_id': 1004,
  'loser_id': 1032,
  'winner_char': 'melee/jigglypuff',
  'loser_score': 0,
  'stage': "Yoshi's Story"},
 {'loser_char': 'melee/peach',
  'winner_score': 2,
  'winner_id': 1004,
  'loser_id': 1032,
  'winner_char': 'melee/jigglypuff',
  'loser_score': 0,
  'stage': "Yoshi's Story"}]

In [ ]:
# Making player vs player data

In [42]:
pd.DataFrame?

Init signature:
pd.DataFrame(
    data=None,
    index: 'Axes | None' = None,
    columns: 'Axes | None' = None,
    dtype: 'Dtype | None' = None,
    copy: 'bool | None' = None,
) -> 'None'
Docstring:     
Two-dimensional, size-mutable, potentially heterogeneous tabular data.

Data structure also contains labeled axes (rows and columns).
Arithmetic operations align on both row and column labels. Can be
thought of as a dict-like container for Series objects. The primary
pandas data structure.

Parameters
----------
data : ndarray (structured or homogeneous), Iterable, dict, or DataFrame
    Dict can contain Series, arrays, constants, dataclass or list-like objects. If
    data is a dict, column order follows insertion-order. If a dict contains Series
    which have an index defined, it is aligned by its index. This alignment also
    occurs if data is a Series or a DataFrame itself. Alignment is done on
    Series/DataFrame inputs.

    If data is a list of dicts, column order follows 

In [25]:
all_player_ids = list(set(list(players_df['player_id'])))
all_player_ids[:5]
# this is dumb, should just loop over rows in sets
#pairs = list(itertools.combinations(all_player_ids, 2))
pvp_df = pd.DataFrame(columns=['p1_id', 'p2_id', 'sets_played', 'p1_sets_won', 'p2_sets_won', 'game_ratios'])
pvp_df.head()

,p1_id,p2_id,sets_played,p1_sets_won,p2_sets_won,game_ratios


In [67]:
for index, row in sets_df.iterrows():
    print(f"p1_id = {row['p1_id']}, p2_id = {row['p2_id']}")
    df_row = pvp_df[((pvp_df['p1_id'] == row['p1_id']) & (pvp_df['p2_id'] == row['p2_id'])) | ((pvp_df['p1_id'] == row['p2_id']) & (pvp_df['p1_id'] == row['p2_id']))]
    print(f"df_row = {df_row}")
    if len(df_row) == 0: # no row exists; add new row
        print("adding new row")
        new_row = [row['p1_id'], row['p2_id'], 0, 0, 0, []]
        print(f"new_row = {new_row}")
        new_row_dict = {k:v for k,v in zip(pvp_df.columns, new_row)}
        print(f"new_row_dict = {new_row_dict}")
        new_row_frame = pd.DataFrame([new_row_dict])
        print(f"new_row_frame = {new_row_frame}")
        pvp_df = pd.concat([pvp_df, new_row_frame])
        print(f"pvp_df = {pvp_df}")
    df_row = pvp_df[((pvp_df['p1_id'] == row['p1_id']) & (pvp_df['p2_id'] == row['p2_id'])).any() | ((pvp_df['p1_id'] == row['p2_id']) & (pvp_df['p1_id'] == row['p2_id']))]
    df_row['sets_played'] += 1
    print(f"winner_id = {row['winner_id']}")
    #print(f"bool test1 {row['winner_id'] == df_row['p1_id']}")
    #print(f"bool test2 {row['winner_id'] == df_row['p2_id']}")
    if (row['winner_id'] == df_row['p1_id']).all():
        df_row['p1_sets_won'] += 1
    else:
        assert row['winner_id'] == df_row['p2_id']
        df_row['p2_sets_won'] += 1
    df_row['game_ratios'] = pd.concat([df_row['game_ratios'], pd.Series([row['p1_score']/(row['p1_score']+row['p2_score'])])])

p1_id = 5620, p2_id = Chillin
df_row = Empty DataFrame
Columns: [p1_id, p2_id, sets_played, p1_sets_won, p2_sets_won, game_ratios]
Index: []
adding new row
new_row = ['5620', 'Chillin', 0, 0, 0, []]
new_row_dict = {'p1_id': '5620', 'p2_id': 'Chillin', 'sets_played': 0, 'p1_sets_won': 0, 'p2_sets_won': 0, 'game_ratios': []}
new_row_frame =   p1_id    p2_id  sets_played  p1_sets_won  p2_sets_won game_ratios
0  5620  Chillin            0            0            0          []
pvp_df =   p1_id    p2_id sets_played p1_sets_won p2_sets_won game_ratios
0  5620  Chillin           0           0           0          []
winner_id = 5620


ValueError: cannot reindex on an axis with duplicate labels

In [55]:
pvp_df.head()

,p1_id,p2_id,sets_played,p1_sets_won,p2_sets_won,game_ratios


## A variation on the other character matchup

In short, the other character vs character matchup that I made relies on collecting all, for example, fox vs fox matches and computing rankings for those. (Seeing how fox players compare to each other). It then uses those elos to compute the elo of someone playing, for example, yoshi vs all fox players.

This variation is different. It uses the *general* elo of every player to compute character matchup elos instead.

In [13]:
# Assumes dataset_generation/game_data_extractor.ipynb was run
game_data_df = pd.read_pickle(data_path + 'individual_game_data.pkl')

In [ ]:
all_characters = list(set(list(game_data_df['p1_char'].unique()) + list(game_data_df['p1_char'].unique())))
all_characters

## Compute the rankings for player/char/char

In [ ]:
# Good for testing
# 1021/yoshi - aMSa
# 19554/fox - Cody

# First, we build the set of rounds with one player and one opponent.
# Each original row in game_data_df will contribute twice, with each player swapping the above roles.

# For convenience, we can restrict our attention to players who actually have a reasonable amount of data with a certain character
MIN_GAMES = 1

game_players_series = pd.concat([game_data_df['p1_id'], game_data_df['p2_id']])
total_games = game_players_series.value_counts()
regular_players = total_games[total_games >= MIN_GAMES]

print("{0} regular player/char combos".format(len(regular_players.index)))

# Lots of memory usage. Let's just reduce down to what we need.
rounds_df = game_data_df[['p1_id', 'p2_id', 'winner_id', 'p1_char', 'p2_char', 'winner_char', 'start', 'end']]

# Each row should contribute twice, swapping 'player' and 'opponent'
df_p1 = rounds_df[rounds_df['p1_id'].apply(lambda x: x in regular_players.index)].copy()
df_p1.rename(columns={'p1_id': 'player_id', 'p2_id': 'opponent_id',
                      'p1_char': 'player_char', 'p2_char': 'opponent_char'}, inplace=True)
df_p1['outcome'] = (df_p1['winner_id'] == df_p1['player_id']).astype(int)
df_p1 = df_p1[['player_id', 'opponent_id', 'player_char', 'opponent_char', 'outcome', 'start', 'end']]
    
df_p2 = rounds_df[rounds_df['p2_id'].apply(lambda x: x in regular_players.index)].copy()
df_p2.rename(columns={'p2_id': 'player_id', 'p1_id': 'opponent_id',
                      'p2_char': 'player_char', 'p1_char': 'opponent_char'}, inplace=True)
df_p2['outcome'] = (df_p2['winner_id'] == df_p2['player_id']).astype(int)
df_p2 = df_p2[['player_id', 'opponent_id', 'player_char', 'opponent_char', 'outcome', 'start', 'end']]

rounds_df = pd.concat([df_p1, df_p2], ignore_index=True)

# Save some memory - these are probably huge.
del df_p1
del df_p2

In [ ]:
rounds_df

In [ ]:
player_ratings_df = pd.read_pickle(data_path + 'overall_players_ranking_new_weekly.pkl')
player_rds_df = pd.read_pickle(data_path + 'overall_players_rds_new_weekly.pkl')

# Returns Rating, RD, and a bool for (actually found = True, default values = False)
def get_opponent_elo_rd(row):
    # Not in our main list of players
    if row['opponent_id'] not in player_ratings_df.columns:
        return (1500.0, 350.0, False)

    # No old enough data
    if player_ratings_df.index[0] > row['start']:
        return (1500.0, 350.0, False)

    # We can take advantage of the fact that the index of player_ratings is always in regular intervals.
    start_date = player_ratings_df.index[0]
    interval = player_ratings_df.index[1] - player_ratings_df.index[0] # I guess we're assuming at least two entries?

    newest_index = int((row['start'] - start_date) / interval)

    # Might actually be out of bounds on the data we have,
    # i.e. 'start' might be well beyond the dates we have data on.
    # In this case, just use the newest piece of data.
    if newest_index >= len(player_ratings_df.index):
        newest_index = len(player_ratings_df.index) - 1

    return (player_ratings_df.iloc[newest_index][row['opponent_id']], player_rds_df.iloc[newest_index][row['opponent_id']], True)

rounds_df['result'] = rounds_df.apply(get_opponent_elo_rd, axis=1)

rounds_df['opponent_rating'] = rounds_df['result'].apply(lambda x: x[0])
rounds_df['opponent_rd']     = rounds_df['result'].apply(lambda x: x[1])
rounds_df['opponent_found']  = rounds_df['result'].apply(lambda x: x[2])

rounds_df.drop(columns=['result'], inplace=True)

rounds_df

In [17]:
# Compute weekly intervals to group by, quite easily.
start_date = datetime.datetime(2015,1,1)
interval = player_ratings_df.index[1] - player_ratings_df.index[0]

# "Copy of a slice" nonsense, this should fix it.
rounds_df = rounds_df.copy()

# Round up, as this computes the date that receives this elo update.
rounds_df['end_index'] = rounds_df['end'].apply(lambda x: math.ceil((x - start_date) / interval))

In [ ]:
# Group by player/character (pc_combo), opponent character, week index
rounds_df['pc_combo'] = rounds_df['player_id'] + '/' + rounds_df['player_char']

grouped_df = rounds_df[['pc_combo', 'opponent_char', 'end_index',
                        'opponent_rating', 'opponent_rd', 'outcome']].groupby(['pc_combo', 'opponent_char', 'end_index']).agg({
        'opponent_rating': list,
        'opponent_rd': list,
        'outcome': list
    }).reset_index()

grouped_df['player_char_char'] = grouped_df['pc_combo'] + '/' + grouped_df['opponent_char']
grouped_df.drop(columns=['pc_combo', 'opponent_char'], inplace=True)
grouped_df

In [20]:
# Actually start computing elos for player/char/char combos.
# TODO: This is REALLY slow. Optimize!

# To deal with inlcude_groups=True being deprecated and disallowed soon,
# let's just create a copy of this column
grouped_df['pcc_duplicate'] = grouped_df['player_char_char']

# Parallelization, cause this be SLOW
hyperthreading = True
n_jobs = multiprocessing.cpu_count() // 2 if hyperthreading else multiprocessing.cpu_count()

# Split into separate dataframes and save in separate files.
# This lets us easily run a multiprocessing script later on them.
unique_pcc_combos = list(grouped_df['player_char_char'].unique())
split_pcc_combos = [] # List of lists to filter by

for i in range(0, n_jobs):
    # First n-1 lists will have this length.
    # Last one will have the remainder.
    # This isn't the most even split, but it gets the job done.
    default_length = len(unique_pcc_combos) // n_jobs

    if i != n_jobs - 1:
        split_pcc_combos += [unique_pcc_combos[i*default_length : (i+1)*default_length]]
    else:
        split_pcc_combos += [unique_pcc_combos[i*default_length : ]]

for i, split in enumerate(split_pcc_combos):
    filter = grouped_df['player_char_char'].isin(split)
    split_grouped_df = grouped_df[filter]
    split_grouped_df.to_pickle(data_path + 'char_vs_char_player_rankings_weekly_alt2_temp_' + str(i) + '.pkl')

# grouped_df.groupby('player_char_char').progress_apply(compute_pcc_elo, include_groups=False)

## You should now run the separate multiprocessing scripts